In [35]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

In [4]:
dataset = fetch_20newsgroups(subset='all', 
                             data_home='./sklearn_data', 
                             remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [15]:
documents_df = pd.DataFrame({'document': documents})
documents_df

,document
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...
...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
18842,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...


In [27]:
def clean_documents(df):
    df['document_cleaned'] = pd.Series(df.document)
    df['document_cleaned'] = (df['document_cleaned']
                              .pipe(lambda x: x.str.lower())  # make all letters lowercase 
                              .pipe(lambda x: x.str.replace("\S*@\S*\s?", "", regex=True))  # remove email addresses
                              .pipe(lambda x: x.str.replace("[^a-zA-Z#]", " ", regex=True)))  # remove non-alphabetical characters

In [28]:
clean_documents(documents_df)
documents_df

,document,document_cleaned
0,\n\nI am sure some bashers of Pens fans are pr...,i am sure some bashers of pens fans are pret...
1,My brother is in the market for a high-perform...,my brother is in the market for a high perform...
2,\n\n\n\n\tFinally you said what you dream abou...,finally you said what you dream about me...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think it s the scsi card doing the dma tran...
4,1) I have an old Jasmine drive which I cann...,i have an old jasmine drive which i cann...
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,dn from david nye dn a neurology dn cons...
18842,\nNot in isolated ground recepticles (usually ...,not in isolated ground recepticles usually a...
18843,I just installed a DX2-66 CPU in a clone mothe...,i just installed a dx cpu in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...,wouldn t this require a hyper sphere in s...


In [37]:
tfidf = TfidfVectorizer(stop_words='english')
dtm = tfidf.fit_transform(documents_df.document_cleaned)
dtm.shape

(18846, 84551)

In [36]:
stopwords.words('english')

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/daniel/nltk_data'
    - '/home/daniel/miniconda3/envs/LSI_dev/nltk_data'
    - '/home/daniel/miniconda3/envs/LSI_dev/share/nltk_data'
    - '/home/daniel/miniconda3/envs/LSI_dev/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
